In [12]:
import numpy as np
import pandas as pd
import time
import datetime
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchFrameException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import warnings
warnings.filterwarnings('ignore')

In [13]:
city = ['DEL','BLR','BOM','PNQ','HYD','AMD','BUP','BHO','GWL','MYQ','CCU','IXZ','IXG','NAG','SXR','BHJ']
src_dst=[]
for rnd in range(20):
    _source=random.choice(city)
    _destination=random.choice(city)
    if _source!=_destination:
        srcDst=_source+'-'+_destination
        src_dst.append(srcDst)

In [14]:
today = datetime.datetime.strftime(datetime.datetime.today() , '%d/%m/%Y')
_60daysfromnow=[]

for i in range(60):
    oneDayLater = datetime.datetime.today() + datetime.timedelta(days = i)
    _60daysfromnow.append(datetime.datetime.strftime(oneDayLater , '%Y%m%d'))

In [15]:
uniquesoruceanddestination=set(src_dst)

In [17]:
chrome_options = webdriver.ChromeOptions()
agents = ["Firefox/66.0.3","Chrome/95.0.4638.54","Edge/16.16299"]
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()

airlines=[]
src=[]
dst=[]
duration=[]
prices=[]
departure=[]
arival=[]
traveldate=[]

for date in _60daysfromnow:
    for sd in uniquesoruceanddestination:
        searchitineraray=sd+'-'+date
        
        url='https://www.goibibo.com/flights/air-'+searchitineraray+'--1-0-0-E-D/?exp=v1'
        driver.get(url)
        time.sleep(2)
        
        try:
            airline=driver.find_elements_by_xpath('//*[@class="font13 padL5 black"]')
            for ar in airline:
                airlines.append(ar.text)
            
            for dt in range(len(airline)):
                traveldate.append(date)
                
            source=driver.find_elements_by_xpath('//*[@class="font12 grey"]//span')
            for srl in source:
                src.append(srl.text)
                
            destination=driver.find_elements_by_xpath('//*[@class="truncate font12 grey"]//span')
            for d in destination:
                dst.append(d.text)
                
            dur=driver.find_elements_by_xpath('//*[@class="srp-card-uistyles__DurTime-sc-3flq99-16 bjQlFP fb padT10"]')
            for drt in dur:
                duration.append(drt.text)
                
            price=driver.find_elements_by_xpath('//*[@class="srp-card-uistyles__Price-sc-3flq99-17 gqEhhU alignItemsCenter dF fb lh1 padT5"]')
            for p in price:
                prices.append(p.text)
                
            depart=driver.find_elements_by_xpath('//*[@class="srp-card-uistyles__Time-sc-3flq99-15 hFzJls padT10 fb"]')
            for dep in depart:
                departure.append(dep.text)
                
            arvl=driver.find_elements_by_xpath('//*[@class="srp-card-uistyles__Time-sc-3flq99-15 hFzJls fb  padT10"]')
            for arv in arvl:
                arival.append(arv.text)
                
        except NoSuchElementException:
            pass

In [18]:
source=src[1::3]
destination=dst[1::3]

In [20]:
print(len(airlines))
print(len(source))
print(len(destination))
print(len(arival))
print(len(departure))
print(len(duration))
print(len(prices))
print(len(traveldate))

3350
3350
3350
3350
3350
3350
3350
3350


In [25]:
dateOfJourney=[]
for i in traveldate:
    yy=i[:4]
    mm=i[4:6]
    dd=i[6:]
    
    dmy=dd+'-'+mm+'-'+yy
    dateOfJourney.append(dmy)

In [30]:
fares=[]

In [32]:
 for p in prices:
        s,e=p.split(',')
        f=s+e
        fares.append(f)

In [37]:
goibibo=pd.DataFrame({
    'Airline':airlines,
    'Date of Journey':dateOfJourney,
    'Source':source,
    'Departure':departure,
    'Duration':duration,
    'Arival':arival,
    'Destination':destination,
    'Fare':fares
})

In [38]:
goibibo

Airline Date of Journey     Source Departure Duration Arival  \
0        IndiGo      01-11-2021     Mumbai     22:15  10h 55m  09:10   
1        IndiGo      01-11-2021     Mumbai     22:55  10h 40m  09:35   
2     Air India      01-11-2021     Mumbai     21:30   9h 50m  07:20   
3        IndiGo      01-11-2021  Ahmedabad     22:40   9h 30m  08:10   
4        IndiGo      01-11-2021  Ahmedabad     23:40   9h 30m  09:10   
...         ...             ...        ...       ...      ...    ...   
3345   Go First      30-12-2021  Hyderabad     05:15  13h 45m  19:00   
3346     IndiGo      30-12-2021  Bengaluru     06:55   2h 15m  09:10   
3347     IndiGo      30-12-2021  Bengaluru     22:40   9h 40m  08:20   
3348     IndiGo      30-12-2021  Bengaluru     12:40   8h 30m  21:10   
3349  Air India      30-12-2021  Bengaluru     06:45  14h 15m  21:00   

     Destination  Fare  
0         Bhopal  7712  
1         Bhopal  8096  
2         Bhopal  9828  
3         Bhopal  9970  
4         Bhopal  8054  
...          ...   ...  
3345        Pune  3066  
3346      Bhopal  3708  
3347      Bhopal  4364  
3348      Bhopal  4574  
3349      Bhopal  4780  

[3350 rows x 8 columns]

In [39]:
goibibo.to_csv('goibibo.csv')